In [ ]:
import os
import tensorflow as tf

train_dir='../dataset\\cifar_10_small\\train'
test_dir='../dataset\\cifar_10_small\\test'

train_aeroplane_dir= os.path.join(train_dir,'aeroplane')
train_bird_dir=os.path.join(train_dir,'bird')
train_car_dir= os.path.join(train_dir,'car')
train_cat_dir=os.path.join(train_dir,'cat')

test_aeroplane_dir= os.path.join(test_dir,'aeroplane')
test_bird_dir=os.path.join(test_dir,'bird')
test_car_dir= os.path.join(test_dir,'car')
test_cat_dir=os.path.join(test_dir,'cat')

print('훈련용 aeroplane 이미지 전체 개수:', len(os.listdir(train_aeroplane_dir)))
print('훈련용 bird 이미지 전체 개수:', len(os.listdir(train_bird_dir)))
print('훈련용 car 이미지 전체 개수:', len(os.listdir(train_car_dir)))
print('훈련용 cat 이미지 전체 개수:', len(os.listdir(train_cat_dir)))

print('테스트용 aeroplane 이미지 전체 개수:', len(os.listdir(test_aeroplane_dir)))
print('테스트용 bird 이미지 전체 개수:', len(os.listdir(test_bird_dir)))
print('테스트용 car 이미지 전체 개수:', len(os.listdir(test_car_dir)))
print('테스트용 cat 이미지 전체 개수:', len(os.listdir(test_cat_dir)))

# parameter
batch_size=20
learning_rate=0.0001
epochs=5

## DATA를 만듬
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

## 만든데이터를 불러와서 파씽함.
train_generator = train_datagen.flow_from_directory(
        directory=train_dir,         # 타깃 디렉터리
        target_size=(32, 32),      # 모든 이미지를 32 × 32 크기로 바꿉니다
        batch_size=batch_size,
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode ='rgb',
        shuffle='True',
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical', # binary, categorical , sparse , input 등이 있음
        subset='training') # ImageDataGenerator 옵션에서  validation_split=0.2에서 1을 뺀 0.8만큼 train data 생성

validataion_generator = train_datagen.flow_from_directory(
        directory=train_dir,         # 타깃 디렉터리
        target_size=(32, 32),      # 모든 이미지를 32 × 32 크기로 바꿉니다
        batch_size=batch_size,
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode ='rgb',
        shuffle='True',
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical', # binary, categorical , sparse , input 등이 있음
        subset='validation')# ImageDataGenerator 옵션에서  validation_split=0.2 기준만큼 validataion data 생성

## class의 인덱스를 확인.
print(train_generator.class_indices)
print(train_generator.classes)

test_generator = test_datagen.flow_from_directory(
        directory=test_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        shuffle='True',
        interpolation='bilinear',  ## resize시 interpolatrion 기법
        color_mode='rgb',
        class_mode='categorical') #categorical

## 파씽한 데이터의 배치사이즈 확인하기
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    print('class :',train_generator.class_indices)
    break


## 모델
input_Layer = tf.keras.layers.Input(shape=(32,32,3))
x=tf.keras.layers.Conv2D(16,(3,3),strides=1, activation='relu')(input_Layer)
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.MaxPool2D((2,2))(x)
x=tf.keras.layers.Conv2D(32,(3,3),strides=1,activation='relu')(x)
x=tf.keras.layers.MaxPool2D((2,2))(x)
# x=tf.keras.layers.Conv2D(128,(3,3),strides=1,activation='relu')(x)
# x=tf.keras.layers.Conv2D(64,(3,3),strides=1,activation='relu')(x)
# x=tf.keras.layers.MaxPool2D((2,2))(x)
x=tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(128, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()

loss_function=tf.keras.losses.categorical_crossentropy
optimize=tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
metric=tf.keras.metrics.categorical_accuracy
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

# callbacks_list= [tf.keras.callbacks.TensorBoard(log_dir='log_dir', histogram_freq=1, write_images='True', write_graph='True')]

## generator는 입력과 타깃의 배치를 끝없이 반환한다.
## 데이터가 끝없이 생성되기 때문에 모델에 하나의 에포크를 정의할때 제너레이터로부터 얼만큼 많은 샘플을 뽑을지 전달해야함
## steps_pr_epoch 가 100이면 위에서 선언된 배치 수만큼의 인풋/아웃풋데이터가 생성되어 학습이 된다.
##즉, 배치가 20이면 20의 배치인 데이터가 100번만큼 생성되어 학습한다. 즉, 20의 배치 데이터를 100번 학습완료하면 1에포크
## 단, 20의 배치데이터를 생성할때마다 랜덤적으로 생성한다.
## 일반적으로 배치사이즈/전체 데이터 길이를 steps_per_epoch로 설정한다.
history = model.fit(
      train_generator,
      steps_per_epoch=((len(os.listdir(train_aeroplane_dir)) + len(os.listdir(train_bird_dir))+len(os.listdir(train_car_dir))+len(os.listdir(train_cat_dir)))*0.8)//batch_size,
      epochs=epochs,
      validation_data=validataion_generator,
      # callbacks=callbacks_list,
      validation_freq=1
)

print(" test acc : ", model.evaluate(test_generator)[1])

model.save('cifar10_categorical_classification.hdf5')

import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()